# Test

In [ ]:
import torch
import os
import joblib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, max_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import DataLoader, TensorDataset

# ==================== 数据准备 =====================
def prepare_test_data_ML(test_df, features_cols, scaler_x, 
                      window_size=5, prediction_step=0):
    imputer = SimpleImputer(strategy="mean")
    X_test = imputer.fit_transform(test_df[features_cols])
    X_test_scaled = scaler_x.transform(X_test)
    test_sequences_X = []
    max_start_idx = len(X_test_scaled) - window_size - prediction_step
    for i in range(max_start_idx + 1):
        window = X_test_scaled[i:i + window_size]
        test_sequences_X.append(window)
    test_sequences_X = np.array(test_sequences_X)
    return test_sequences_X

def get_predictions_ML(test_sequences_X, model, scaler_y):
    predictions_soh = model.predict(test_sequences_X.reshape(test_sequences_X.shape[0], -1))
    predictions_soh = scaler_y.inverse_transform(predictions_soh.reshape(-1, 1)).flatten()
    return predictions_soh

def test_run_ML(file_path, model_name):
    scaler_x = joblib.load("scaler_x_all.pkl")
    scaler_y = joblib.load("scaler_y_all.pkl")
    features_cols = [
        'mileage', 'temp_ch', 'temp_rest', 'curr_0', 'volt_end', 'volt_1', 'rest_volt1_mean', 'dcr_1', 'dV_1', 'volt_10', 'dcr_10', 'dV_10',
        'vf001_mo', 'vf002_mo', 'vf003_mo', 'vf004_mo', 'zf001_mo', 'zf002_mo', 'zf003_mo', 'zf004_mo', 'zf001_ph',
        'rest_volt1_std', 'rest_volt1_range', 'rest_dcr10_std', 'rest_volt10_range', 'zf004_mo_std', 'zf002_mo_range', 'zf003_mo_range',
        'zf001_ph_std', 'zf002_ph_std', 'zf003_ph_std', 'zf004_ph_std'
    ]
    model_dir = os.path.join('model',model_name)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model_path = os.path.join(model_dir,'final_model.pth')
    model = joblib.load(model_path)
    window_size = 10
    df = pd.read_excel(file_path)
    test_data = prepare_test_data_ML(df, features_cols, scaler_x, window_size, prediction_step=0)
    real_soh = df['soh'].iloc[window_size-1:] * 100
    mile = df['mileage'].iloc[window_size-1:] / 1000
    predictions_soh = get_predictions_ML(test_data, model, scaler_y)
    return predictions_soh

In [ ]:
model_name = 'RF'
# model_name = 'XGB'
# model_name = 'LGB'

# -------------------------------------------------------------
file_path = r"data\data_test\NMC02_fast_feature.xlsx"
test_run_ML(file_path, model_name)